# 03-filter_nodes_edges.ipynb
- Remove edges with nodes with no umls type or label
- Remove the following predicates: ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 
'same_as','OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES','AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']

In [1]:
import os
import pickle
%pylab
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict
from tqdm import tqdm

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
DATA  = 'data/'
EDGES4_CSV = DATA+"edges4.csv"
NODES_BLM_CSV = DATA+"nodes_blm.csv"
EDGES_FILTERED_CSV = DATA+"edges_filtered.csv"
NODES_FILTERED_CSV = DATA+"nodes_filtered.csv"

In [3]:
edges = pd.read_csv(EDGES4_CSV, index_col=0)
edges.head()

/Users/richard/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID,NEG
588,C1412045,ADMINISTERED_TO,C0007634,24096582,False
589,C1412045,ADMINISTERED_TO,C0030705,22577025,False
590,C1412045,AFFECTS,C0005935,29798367,False
591,C1412045,AFFECTS,C0020291,6298464,False
592,C1412045,AFFECTS,C0028754,19789049,False


In [4]:
nodes = pd.read_csv(NODES_BLM_CSV, index_col=0)
nodes.head()

,ID,LABEL,umls_type,umls_type_label,blm_category
0,C3842672,Day 7,T033,Finding,disease_or_phenotypic_feature
6,C0020684,Hypoxanthine,T123|T109,Biologically Active Substance|Organic Chemical,chemical_substance
7,C0853225,INR Increased,T033,Finding,disease_or_phenotypic_feature
8,C1513022,Mature Centriole,T026,Cell Component,cell_component
10,C0267183,Hourglass stricture or stenosis of stomach,T047,Disease or Syndrome,disease_or_phenotypic_feature


In [5]:
print(len(nodes))
print(len(edges))

168082
23932681


In [6]:
# remove edges with no nodes
print(len(edges))
edges = edges[edges.SUBJECT_CUI.isin(nodes.ID) & edges.OBJECT_CUI.isin(nodes.ID)]
print(len(edges))

23932681
16220205


In [7]:
vc = edges.PREDICATE.value_counts()
vc

INTERACTS_WITH      2220070
LOCATION_OF         2105454
COEXISTS_WITH       1780772
AFFECTS             1614278
STIMULATES          1307942
INHIBITS            1106486
ASSOCIATED_WITH      939793
CAUSES               759205
PART_OF              751794
AUGMENTS             548271
DISRUPTS             534850
TREATS               484268
PRODUCES             480627
compared_with        368322
PREDISPOSES          269523
ISA                  237589
PREVENTS             145866
PROCESS_OF           141374
higher_than          112266
ADMINISTERED_TO       66451
PRECEDES              52602
CONVERTS_TO           48751
COMPLICATES           37896
MANIFESTATION_OF      36672
same_as               24564
lower_than            21456
USES                  15793
DIAGNOSES              4840
OCCURS_IN              1197
MEASURES               1048
METHOD_OF               144
MEASUREMENT_OF           38
PREP                      3
Name: PREDICATE, dtype: int64

In [8]:
def drop_edges(df, drop_edges):
    idx = df.query('PREDICATE in @drop_edges').index
    df.drop(idx, inplace=True)

In [9]:
# remove edges we don't need
print(len(edges))
remove_edges = ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 'same_as',
               'OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES',
               'AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']
drop_edges(edges, remove_edges)
print(len(edges))

16220205
14877631


In [10]:
# remove nodes with no edges
print(len(nodes))
nodes = nodes[nodes.ID.isin(set(list(edges.SUBJECT_CUI) + list(edges.OBJECT_CUI)))]
print(len(nodes))

168082
156759


In [11]:
vc = edges.PREDICATE.value_counts()
vc

INTERACTS_WITH      2220070
LOCATION_OF         2105454
COEXISTS_WITH       1780772
AFFECTS             1614278
STIMULATES          1307942
INHIBITS            1106486
ASSOCIATED_WITH      939793
CAUSES               759205
PART_OF              751794
DISRUPTS             534850
TREATS               484268
PRODUCES             480627
PREDISPOSES          269523
ISA                  237589
PREVENTS             145866
PRECEDES              52602
CONVERTS_TO           48751
MANIFESTATION_OF      36672
MEASURES               1048
MEASUREMENT_OF           38
PREP                      3
Name: PREDICATE, dtype: int64

In [12]:
edges.to_csv(EDGES_FILTERED_CSV, index=False)
nodes.to_csv(NODES_FILTERED_CSV, index=False)